<a href="https://colab.research.google.com/github/german-suslin/test_task/blob/main/test_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U spacy
!python -m spacy download ru_core_news_lg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2022-09-04 21:10:21.573828: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 513.4 MB 15 kB/s 
     |████████████████████████████████| 55 kB 3.1 MB/s 
     |████████████████████████████████| 8.2 MB 13.4 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=6108313cac68b47234ecafe39be2ae65fde9e57e8d22f535cb60d172db9b9f65
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')


In [ ]:
import spacy # библиотека для работы с ner
import pandas as pd
import re
import ru_core_news_lg # Датасет для ner
nlp = ru_core_news_lg.load() # подгружаем модель для определённого датасета
# nlp = spacy.load('ru_core_news_lg')
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
filepath = '/content/drive/MyDrive/Work Bewise_ai/test_data.csv'
df = pd.read_csv(filepath)
# Функция извлекает имена собственные из текста
def name_recognition(text:str)->list: 
  # Возвращает список именнованых функций
  documents = nlp(text)
  names = []
  for named_entity in documents.ents:
    if named_entity.label_ == "PER":
      names.append(named_entity.text)
  return names

# Функция приводит слова к нормальной форме в нижнем регистре
def take_lemmas(text:str)->list:
  document = nlp(text)
  lemmas_list = []
  for token in document:
    lemmas_list.append(token.lemma_.lower())
  return lemmas_list

# Функция извлечения признаков
def parse_message(message:str):
  '''
  Функция принимает сообщение и возвращает 5 значений
  greet_flag - флаг приветствия - поприветствовал ли менеджер
  introduce_flag - флаг представления - представился ли менеджер
  name_text - имя менеджера, если не было найдено - unknown
  company_text - название компании, если не было найдено - unknown
  goodbye_flag - флаг прощания - попрощался ли менеджер
  '''
  greet_flag = False
  introduce_flag = False
  name_text = 'unknown'
  company_text = 'unknown'
  goodbye_flag = False

  # Паттерны поиска для регулярных выражений
  greetings_pattern = [r'(зд(\w)+вовать)+', r'(привет)+', r'(доб(\w)*й.д(\w)*ь)+']
  company_pattern = r'(ком(\w)*ния)+'
  name_pattern = r'(ме(\w)*я)+.*(зв(\w)*ть)+'
  goodbye_pattern = [r'(в\w* *доб\w+ого)+', r'(до *\w*ния)+', r'((всего|весь) *х\w+его)+']

  message = message.lower() # переводим в нижний регистр
  lemmas = take_lemmas(message) # лемманизируем сообщение
  lemmas_text = ' '.join(map(str, lemmas)) # соединяем леммы в один текст


  # Проходим по паттернам приветствия, если находим совпадение, выставляем флаг
  for greet in greetings_pattern:
    if re.search(greet, lemmas_text):
      greet_flag = True

  # Ищем по паттерну компании (для расширенного датасета потребуется составить
  # список компаний, полагаю, либо расшифровку делать с учётом регистра, т.к. 
  # почти все ner модели не работают с нижним регистром для организаций)
  if re.search(company_pattern + ' \w+ *' + ' *бизнес', lemmas_text):
    company = re.search(company_pattern +' \w+ *' + ' *бизнес ', lemmas_text)
    company_text = company.group()

  # Ищем ner per - личные имена, если есть - выставляем флаг, записываем имя
  if re.search(name_pattern, lemmas_text):
    people = name_recognition(message)
    if len(people)>0:
      for name in people:
        if not re.search(name, company.group()): # использованная модель иногда
                                                 # выделяла "китобизнес", поэтому здесь проверка
          introduce_flag = True
          name_text = name
  # Проходим по паттернам прощания и выставляем флаг
  for goodbye in goodbye_pattern:
    if re.search(goodbye, message):
      goodbye_flag = True
      break # Если срабатывает несколько флагов - чтобы прервать цикл при первом срабатывании
  return greet_flag, introduce_flag, goodbye_flag, name_text, company_text

# Функция извлечения реплик приветствия, представления, прощания, имени менеджера и названия компании
def get_parameters_from_df(filepath:str, return_df:bool=False):
  '''
  Функция принимает путь до файла csv (.../test/test.csv)
  Возвращает либо датасет с колонками флагов, имени и названия компании
  либо 4 датасетов 
  датасет с репликами приветствия
  датасет с репликами представления (и именем менеджера соответственно)
  датасет с репликами прощания
  датасет с именем названием компании
  '''
  df = pd.read_csv(filepath)
  df_manager = df[df['role']=='manager']
  greet_flag_list = []
  introduce_flag_list = []
  goodbye_flag_list = []
  name_text_list = []
  company_text_list = []

  for i, row in df.iterrows():
    greet_flag, introduce_flag, goodbye_flag, name_text, company_text = parse_message(row['text'])

    greet_flag_list.append(greet_flag)
    introduce_flag_list.append(introduce_flag)
    goodbye_flag_list.append(goodbye_flag)
    name_text_list.append(name_text)
    company_text_list.append(company_text)

  df['greetings'] = greet_flag_list.copy()
  greet_flag_list.clear()
  df['introduce'] = introduce_flag_list.copy()
  introduce_flag_list.clear()
  df['goodbye'] = goodbye_flag_list.copy()
  goodbye_flag_list.clear()
  df['manager name'] = name_text_list.copy()
  name_text_list.clear()
  df['company'] = company_text_list.copy()
  company_text_list.clear()
  if return_df:
    return df
  output_greetings = df[df['greetings']]
  output_introduce = df[df['introduce']]
  output_goodbye = df[df['goodbye']]
  # output_manager_name = df[df['manager name']!='unknown']
  output_company = df[df['company']!='unknown']
  return output_greetings, output_introduce, output_goodbye, output_company

  
greet, introduce, goodbye, company = get_parameters_from_df(filepath)

In [ ]:
greet

,dlg_id,line_n,role,text,greetings,introduce,goodbye,manager name,company
1,0,1,manager,Алло здравствуйте,True,False,False,unknown,unknown
2,0,2,client,Добрый день,True,False,False,unknown,unknown
109,1,0,client,Да здравствуйте когда заканчивается,True,False,False,unknown,unknown
110,1,1,manager,Алло здравствуйте,True,False,False,unknown,unknown
165,2,1,client,Здравствуйте,True,False,False,unknown,unknown
166,2,2,manager,Алло здравствуйте,True,False,False,unknown,unknown
249,3,0,client,Добрый день,True,False,False,unknown,unknown
250,3,1,manager,Алло дмитрий добрый день,True,False,False,unknown,unknown


In [ ]:
introduce

,dlg_id,line_n,role,text,greetings,introduce,goodbye,manager name,company
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается,False,True,False,ангелина,компания диджитал бизнес
111,1,2,manager,Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается,False,True,False,ангелина,компания диджитал бизнес
167,2,3,manager,Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там,False,True,False,ангелина,компания диджитал бизнес
251,3,2,manager,Добрый меня максим зовут компания китобизнес удобно говорить,False,True,False,максим,компания китобизнес


In [ ]:
goodbye

,dlg_id,line_n,role,text,greetings,introduce,goodbye,manager name,company
96,0,96,client,Потому что это ну вот эти звонки за месяц до окончания,False,False,True,unknown,unknown
108,0,108,manager,Всего хорошего до свидания,False,False,True,unknown,unknown
162,1,53,manager,Угу да вижу я эту почту хорошо тогда исправлю на эту будем ждать ответа всего хорошего,False,False,True,unknown,unknown
163,1,54,manager,До свидания,False,False,True,unknown,unknown
300,3,51,manager,Угу все хорошо да понедельника тогда всего доброго,False,False,True,unknown,unknown
301,3,52,client,Да до свидания,False,False,True,unknown,unknown
335,4,33,manager,Во вторник все ну с вами да тогда до вторника до свидания,False,False,True,unknown,unknown
336,4,34,client,Угу да до свидания,False,False,True,unknown,unknown
479,5,142,manager,Ну до свидания хорошего вечера,False,False,True,unknown,unknown


In [ ]:
name

,dlg_id,line_n,role,text,greetings,introduce,goodbye,manager name,company
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается,False,True,False,ангелина,компания диджитал бизнес
111,1,2,manager,Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается,False,True,False,ангелина,компания диджитал бизнес
167,2,3,manager,Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там,False,True,False,ангелина,компания диджитал бизнес
251,3,2,manager,Добрый меня максим зовут компания китобизнес удобно говорить,False,True,False,максим,компания китобизнес


In [ ]:
company

,dlg_id,line_n,role,text,greetings,introduce,goodbye,manager name,company
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается,False,True,False,ангелина,компания диджитал бизнес
111,1,2,manager,Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается,False,True,False,ангелина,компания диджитал бизнес
167,2,3,manager,Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там,False,True,False,ангелина,компания диджитал бизнес
251,3,2,manager,Добрый меня максим зовут компания китобизнес удобно говорить,False,True,False,максим,компания китобизнес


In [ ]:
df = get_parameters_from_df(filepath, return_df = True)

In [ ]:
df['dlg_id'].unique()

array([0, 1, 2, 3, 4, 5])

In [ ]:
# проверка требования приветствия и прощания для менеджера
df['check_manager_requirement'] = False

for i in df['dlg_id'].unique():
  greeting_req = False
  goodbye_req = False
  current_df = df[df['dlg_id']==i]
  for j, row in current_df.iterrows():
    if current_df.loc[j, 'greetings']:
      greeting_req = True
    if current_df.loc[j, 'goodbye']:
      goodbye_req = True
  if goodbye_req and greeting_req:
    df['check_manager_requirement'][df['dlg_id']==i] = True

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [ ]:
df

,dlg_id,line_n,role,text,greetings,introduce,goodbye,manager name,company,check_manager_requirement
0,0,0,client,Алло,False,False,False,unknown,unknown,True
1,0,1,manager,Алло здравствуйте,True,False,False,unknown,unknown,True
2,0,2,client,Добрый день,True,False,False,unknown,unknown,True
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,False,True,False,ангелина,компания диджитал бизнес,True
4,0,4,client,Ага,False,False,False,unknown,unknown,True
...,...,...,...,...,...,...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,False,False,False,unknown,unknown,False
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,False,False,False,unknown,unknown,False
477,5,140,client,Спасибо спасибо,False,False,False,unknown,unknown,False
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте,False,False,False,unknown,unknown,False
